In [1]:
import torchvision
import torch
from IPython.display import display
import ipywidgets
import traitlets
from robotpal import Camera, bgr8_to_jpeg, Robot
from robotpal.SCSCtrl import TTLServo
import torchvision.transforms as transforms
import cv2
import PIL.Image
import numpy as np
import os
import threading
import time
from datetime import datetime

# --------------------------------------------------------------
# 1. 초기 설정 (고화질 카메라 연결)
# --------------------------------------------------------------
# [핵심] server.py를 수정했으므로 이제 고화질을 받아올 수 있습니다.
CAM_WIDTH = 816
CAM_HEIGHT = 616

camera = Camera.instance(width=CAM_WIDTH, height=CAM_HEIGHT)
robot = Robot()

# 모델 로드
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy_test_12_17.pth', map_location=torch.device('cuda')))
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ... ]

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998
[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)
[System] WebSocket 프로세서 시작


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


현재 모델 가중치는 CPU 메모리에 있습니다. 아래 코드를 실행하여 GPU 장치로 전송합니다.

In [ ]:
TTLServo.servoAngleCtrl(5, 45, 1, 100)

# --------------------------------------------------------------
# 2. 색상 인식 및 고화질 스냅샷 준비 (스레드)
# --------------------------------------------------------------
frame_width = CAM_WIDTH
frame_height = CAM_HEIGHT
colorHSVvalueList = []
max_len = 20
th_flag = True

# UI 위젯
colorlblMax = ipywidgets.Text(description="Max HSV:", value="Wait...")
colorlblMin = ipywidgets.Text(description="Min HSV:", value="Wait...")
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)

# 스냅샷 저장용 전역 변수
latest_annotated_frame = None
frame_lock = threading.Lock()

def colorRecog():
    global latest_annotated_frame, colorHSVvalueList
    
    # [좌표 보정] 해상도가 816px로 커졌으니 이동 값도 키웁니다.
    # (224px일 때 60이었던 비율 -> 816px에서는 약 220~230)
    move_x = -73
    move_y = 220
    
    while th_flag:
        if camera.value is None:
            time.sleep(0.1)
            continue
            
        # 1. 고화질 원본 복사 (server.py 수정 덕분에 816x616 이미지가 들어옴)
        image = camera.value.copy()
        
        # 2. 색상 분석 및 그리기 (고화질 이미지 위에 수행)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv = cv2.blur(hsv, (15, 15))
        
        # 좌표 계산
        center_x = int(frame_width / 2) + move_x
        center_y = int(frame_height / 2) + move_y
        
        center_x = max(0, min(center_x, frame_width - 1))
        center_y = max(0, min(center_y, frame_height - 1))
        
        hsvValue = hsv[center_y, center_x]

        colorHSVvalueList.append(hsvValue)
        if len(colorHSVvalueList) > max_len:
            del colorHSVvalueList[0]
            
        if len(colorHSVvalueList) > 0:
            max_h, max_s, max_v = np.maximum.reduce(colorHSVvalueList)
            min_h, min_s, min_v = np.minimum.reduce(colorHSVvalueList)

            # 박스 크기 키움 (해상도에 맞춰 20 -> 60)
            rect_s = 60
            cv2.rectangle(image,
                          (center_x - int(rect_s/2), center_y - int(rect_s/2)),
                          (center_x + int(rect_s/2), center_y + int(rect_s/2)),
                          (255, 255, 255), 2)
            
            # 글씨 크기 키움 (0.3 -> 0.8) 
            # 해상도가 커져서 0.3으로 하면 안 보입니다.
            cv2.putText(image, f'max:{max_h,max_s,max_v}', (30, 550), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)
            cv2.putText(image, f'min:{min_h,min_s,min_v}', (30, 580), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            
            colorlblMax.value = f"{max_h},{max_s},{max_v}"
            colorlblMin.value = f"{min_h},{min_s},{min_v}"
        
        # ---------------------------------------------------------
        # [핵심 1] 스냅샷 저장용 변수에는 '고화질 원본'을 담습니다.
        # 버튼을 누르면 이 변수에 있는 816x616 이미지가 저장됩니다.
        # ---------------------------------------------------------
        with frame_lock:
            latest_annotated_frame = image
            
        # ---------------------------------------------------------
        # [핵심 2] 화면 표시용은 작게 줄여서 보냅니다.
        # 816px를 매번 보내면 렉 걸리므로 300px로 리사이즈!
        # ---------------------------------------------------------
        preview_img = cv2.resize(image, (300, 300))
        image_widget.value = bgr8_to_jpeg(preview_img)
        
        # CPU 휴식 (주행 스레드 양보)
        time.sleep(0.05)

# 기존 스레드 정리 후 재시작
try:
    th_flag = False
    time.sleep(0.5)
except:
    pass

th_flag = True
thread = threading.Thread(target=colorRecog)
thread.start()

In [ ]:
# --------------------------------------------------------------
# 3. 스냅샷 버튼 기능
# --------------------------------------------------------------
SAVE_DIR = "inspection_snapshots"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

snapshot_btn = ipywidgets.Button(description='HD Snapshot', button_style='warning', icon='camera')
snapshot_msg = ipywidgets.Label(value="준비됨")

def save_snapshot(b):
    with frame_lock:
        if latest_annotated_frame is not None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"Insp_HD_{timestamp}.jpg"
            save_path = os.path.join(SAVE_DIR, filename)
            
            # 저장되는 이미지는 816x616 고화질입니다.
            cv2.imwrite(save_path, latest_annotated_frame)
            snapshot_msg.value = f"저장 완료({latest_annotated_frame.shape[1]}px): {filename}"
        else:
            snapshot_msg.value = "데이터 없음"

snapshot_btn.on_click(save_snapshot)

# --------------------------------------------------------------
# 4. 자율주행 실행 (자동 리사이즈 적용)
# --------------------------------------------------------------
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.VBox([
    image_widget,
    ipywidgets.HBox([colorlblMax, colorlblMin]),
    ipywidgets.HBox([snapshot_btn, snapshot_msg]),
    ipywidgets.HBox([y_slider, speed_slider]),
    ipywidgets.HBox([x_slider, steering_slider])
]))

angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    # [중요] 원본(816px) 이미지를 가져옵니다.
    image = change['new']
    
    # [핵심 3] AI 모델은 작은 이미지(224px)만 먹을 수 있습니다.
    # 따라서 주행 직전에 224x224로 줄여서 넣어줍니다.
    # 이렇게 하면 고화질 카메라를 쓰면서도 주행 속도는 빠릅니다.
    if image is not None:
        image_resized = cv2.resize(image, (224, 224))
        
        xy = model(preprocess(image_resized)).detach().float().cpu().numpy().flatten()
        x = xy[0]
        y = (0.5 - xy[1]) / 2.0
        
        x_slider.value = float(x)
        y_slider.value = float(y)
        
        speed_slider.value = 0.25 # 속도 설정
        
        angle = np.arctan2(x, y)
        pid = angle * 0.2 + (angle - angle_last) * 0.5
        angle_last = angle
        
        steering_slider.value = pid
        
        left_val = max(min(speed_slider.value + steering_slider.value, 1.0), -0.9)
        right_val = max(min(speed_slider.value - steering_slider.value, 1.0), -0.9)
        
        robot.left_motor.value = float(left_val)
        robot.right_motor.value = float(right_val)

execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
import time
camera.unobserve(execute, names='value')
time.sleep(0.1) 
robot.stop()
camera.stop()